In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
import seaborn as sns
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import pos_tag, pos_tag_sents

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet as wn
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
import pandas as pd

# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('sentiwordnet', quiet=True)

True

In [3]:
df=pd.read_csv("one.csv",on_bad_lines="skip")

C:\Users\Kunal Karne\AppData\Local\Temp\ipykernel_16176\2798303258.py:1: DtypeWarning: Columns (38,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("one.csv",on_bad_lines="skip")


In [4]:
df.shape

(150345, 58)

In [5]:
df.columns

Index(['address', 'business_id', 'categories', 'city', 'is_open', 'latitude',
       'longitude', 'name', 'postal_code', 'review_count', 'stars', 'state',
       'attributes_AcceptsInsurance', 'attributes_AgesAllowed',
       'attributes_Alcohol', 'attributes_Ambience', 'attributes_BYOB',
       'attributes_BYOBCorkage', 'attributes_BestNights',
       'attributes_BikeParking', 'attributes_BusinessAcceptsBitcoin',
       'attributes_BusinessAcceptsCreditCards', 'attributes_BusinessParking',
       'attributes_ByAppointmentOnly', 'attributes_Caters',
       'attributes_CoatCheck', 'attributes_Corkage',
       'attributes_DietaryRestrictions', 'attributes_DogsAllowed',
       'attributes_DriveThru', 'attributes_GoodForDancing',
       'attributes_GoodForKids', 'attributes_GoodForMeal',
       'attributes_HairSpecializesIn', 'attributes_HappyHour',
       'attributes_HasTV', 'attributes_Music', 'attributes_NoiseLevel',
       'attributes_Open24Hours', 'attributes_OutdoorSeating',
       '

In [6]:
df['categories'].head(50)

0     Doctors, Traditional Chinese Medicine, Naturop...
1     Shipping Centers, Local Services, Notaries, Ma...
2     Department Stores, Shopping, Fashion, Home & G...
3     Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4                             Brewpubs, Breweries, Food
5     Burgers, Fast Food, Sandwiches, Food, Ice Crea...
6     Sporting Goods, Fashion, Shoe Stores, Shopping...
7                   Synagogues, Religious Organizations
8     Pubs, Restaurants, Italian, Bars, American (Tr...
9     Ice Cream & Frozen Yogurt, Fast Food, Burgers,...
10                 Department Stores, Shopping, Fashion
11           Vietnamese, Food, Restaurants, Food Trucks
12    American (Traditional), Restaurants, Diners, B...
13    General Dentistry, Dentists, Health & Medical,...
14          Food, Delis, Italian, Bakeries, Restaurants
15                    Sushi Bars, Restaurants, Japanese
16    Automotive, Auto Parts & Supplies, Auto Custom...
17    Vape Shops, Tobacco Shops, Personal Shoppi

In [10]:
df.isnull().sum()

address                                    5127
business_id                                   0
categories                                  103
city                                          0
is_open                                       0
latitude                                      0
longitude                                     0
name                                          0
postal_code                                  73
review_count                                  0
stars                                         0
state                                         0
attributes_AcceptsInsurance              144640
attributes_AgesAllowed                   150216
attributes_Alcohol                       107194
attributes_Ambience                      106617
attributes_BYOB                          145906
attributes_BYOBCorkage                   148906
attributes_BestNights                    144662
attributes_BikeParking                    77787
attributes_BusinessAcceptsBitcoin       

In [12]:
df.dropna(subset=['categories'], inplace=True)

In [ ]:
restaurant_keywords = [
    'Restaurants', 'Food', 'Bars', 'Bar', 'Nightlife',
    'Coffee', 'Hotels', 'Sandwiches', 'Breakfast', 
    'Tea', 'Pizza', 'Burgers', 'Wine', 'Vegetarian', 
    'Clubs', 'Kitchen', 'Salad', 'Buffets'
]
# List most frequent words of restaurant categories

In [ ]:
restaurent_df = df[df['categories'].str.contains('|'.join(restaurant_keywords), case=False)]

#make new df for restaurent

In [14]:
restaurent_df.shape

(76193, 58)

In [28]:
saloon_keywords=['Beauty',  'Hair','Spas,','Waxing,','Stylists,', 'Makeup', 'Men\'s', 'Nail', 'Salons', 'Spas', 'Stylists', 'Waxing', 'Women\'s']
# keywords for saloon

In [29]:
saloon_df = df[df['categories'].str.contains('|'.join(saloon_keywords), case=False)]
# make new df for saloon

In [30]:
saloon_df.shape

(16666, 58)

In [19]:
medical_keyword=['Health','Medical,','Medical','Dentists,','Doctors,','Doctors']
# keywords for medical

In [20]:
medical_df = df[df['categories'].str.contains('|'.join(medical_keyword), case=False)]
#new df for medical

In [21]:
medical_df.shape

(12492, 58)

In [24]:
restaurent_df['categories'].head(50)

3     Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4                             Brewpubs, Breweries, Food
5     Burgers, Fast Food, Sandwiches, Food, Ice Crea...
8     Pubs, Restaurants, Italian, Bars, American (Tr...
9     Ice Cream & Frozen Yogurt, Fast Food, Burgers,...
11           Vietnamese, Food, Restaurants, Food Trucks
12    American (Traditional), Restaurants, Diners, B...
14          Food, Delis, Italian, Bakeries, Restaurants
15                    Sushi Bars, Restaurants, Japanese
18    Automotive, Car Rental, Hotels & Travel, Truck...
19                                  Korean, Restaurants
20    Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re...
22               Steakhouses, Asian Fusion, Restaurants
23                                 Restaurants, Italian
24                                Hot Dogs, Restaurants
27                       Restaurants, Japanese, Seafood
28    Cocktail Bars, Bars, Italian, Nightlife, Resta...
29        Pizza, Chicken Wings, Sandwiches, Rest

In [26]:
medical_df['categories'].head(50)

0      Doctors, Traditional Chinese Medicine, Naturop...
13     General Dentistry, Dentists, Health & Medical,...
43     Health & Medical, Beauty & Spas, Laser Hair Re...
52        Specialty Food, Bakeries, Food, Health Markets
57     Ophthalmologists, Eyewear & Opticians, Health ...
74     Health & Medical, Personal Care Services, Beau...
100    Fitness & Instruction, Physical Therapy, Healt...
111    Eyewear & Opticians, Health & Medical, Shoppin...
128    Medical Centers, Doctors, Diagnostic Services,...
137    Pain Management, Doctors, Health & Medical, Os...
139    Food, Organic Stores, Specialty Food, Health M...
146         Doctors, Internal Medicine, Health & Medical
156    Massage, Health & Medical, Medical Spas, Beaut...
166                        Health & Medical, Urgent Care
169    Pharmacy, Health & Medical, Convenience Stores...
174    Weight Loss Centers, Health & Medical, Life Co...
178                      Health & Medical, Chiropractors
184                    Medical 

In [31]:
saloon_df['categories'].head(50)

26     Women's Clothing, Accessories, Children's Clot...
40     Hair Salons, Hair Extensions, Beauty & Spas, W...
43     Health & Medical, Beauty & Spas, Laser Hair Re...
74     Health & Medical, Personal Care Services, Beau...
92                     Fashion, Shopping, Men's Clothing
101    Women's Clothing, Men's Clothing, Accessories,...
123                               Barbers, Beauty & Spas
149                    Fashion, Shopping, Men's Clothing
156    Massage, Health & Medical, Medical Spas, Beaut...
170                                Beauty & Spas, Tattoo
173    Waxing, Hair Removal, Skin Care, Eyelash Servi...
179    Beauty & Spas, Hair Stylists, Hair Extensions,...
181    Venues & Event Spaces, Performing Arts, Arts &...
212    Makeup Artists, Beauty & Spas, Hair Salons, Ha...
214                               Barbers, Beauty & Spas
233     Beauty & Spas, Hair Removal, Day Spas, Skin Care
250     Beauty & Spas, Waxing, Nail Salons, Hair Removal
267                    Fashion,

In [32]:
saloon_df.to_csv('saloon_df.csv', index=False)
# save saloon csv

In [33]:
restaurent_df.to_csv('restaurent_df.csv', index=False)
# save restaurent csv

In [34]:
medical_df.to_csv('medical_df.csv', index=False)
# save medical csv

In [35]:
medical_df.isnull().sum()

address                                    187
business_id                                  0
categories                                   0
city                                         0
is_open                                      0
latitude                                     0
longitude                                    0
name                                         0
postal_code                                  2
review_count                                 0
stars                                        0
state                                        0
attributes_AcceptsInsurance               7109
attributes_AgesAllowed                   12492
attributes_Alcohol                       12289
attributes_Ambience                      12373
attributes_BYOB                          12477
attributes_BYOBCorkage                   12488
attributes_BestNights                    12491
attributes_BikeParking                    9142
attributes_BusinessAcceptsBitcoin        11618
attributes_Bu

In [45]:
medical_df2=medical_df[['address','business_id','categories','city','is_open','latitude','longitude','name','postal_code','review_count','stars','state']]

In [46]:
#medical_df2.shape

(12492, 12)

In [132]:
#medical_df2['business_id'].value_counts()

business_id
Pns2l4eNsfO8kk83dixA6A    1
APr7s73xPQBQlQD90wts3w    1
aG4xsXJS_DgnDQYnflw7WQ    1
JtAp-rbT2e_d6jRwGPSvCA    1
lVmJjNu_WjBsS8TfFOYvWQ    1
                         ..
SEWpBKC0Dp1w4qItJZa_yA    1
sFHkYIL7oQlWcNYOSVWkoA    1
T6a8PIgFxOh9qLNfXTn0bw    1
h3WFtU8bprxlp1YG5oK7Gw    1
fn3ybdsRSrIDpKZTsRuAWg    1
Name: count, Length: 12304, dtype: int64

In [47]:
#medical_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12492 entries, 0 to 150337
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       12305 non-null  object 
 1   business_id   12492 non-null  object 
 2   categories    12492 non-null  object 
 3   city          12492 non-null  object 
 4   is_open       12492 non-null  int64  
 5   latitude      12492 non-null  float64
 6   longitude     12492 non-null  float64
 7   name          12492 non-null  object 
 8   postal_code   12490 non-null  object 
 9   review_count  12492 non-null  int64  
 10  stars         12492 non-null  float64
 11  state         12492 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 1.2+ MB


In [52]:
medical_df2.isnull().sum()

address         187
business_id       0
categories        0
city              0
is_open           0
latitude          0
longitude         0
name              0
postal_code       2
review_count      0
stars             0
state             0
dtype: int64

In [54]:
medical_df2.dropna(inplace=True)

C:\Users\Kunal Karne\AppData\Local\Temp\ipykernel_16176\1170679005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_df2.dropna(inplace=True)


In [56]:
medical_df2.isnull().sum()


address         0
business_id     0
categories      0
city            0
is_open         0
latitude        0
longitude       0
name            0
postal_code     0
review_count    0
stars           0
state           0
dtype: int64

In [55]:
medical_df2.head()

,address,business_id,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1616 Chapala St, Ste 2",Pns2l4eNsfO8kk83dixA6A,"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,0,34.426679,-119.711197,"Abby Rappoport, LAC, CMQ",93101,7,5.0,CA
13,15 N Missouri Ave,jaxMSoInw8Poo3XeMJt8lQ,"General Dentistry, Dentists, Health & Medical,...",Clearwater,1,27.966235,-82.787412,Adams Dental,33755,10,5.0,FL
43,"2 Bala Plz, Ste PL-11",Kq51_lGAgAigqmENITTr-A,"Health & Medical, Beauty & Spas, Laser Hair Re...",Bala Cynwyd,1,40.006870,-75.217262,Bala Better Health,19004,13,4.0,PA
52,"11642 104 Avenue, Unit 642",5BmQX4UVJY19mMtafMg7JA,"Specialty Food, Bakeries, Food, Health Markets",Edmonton,1,53.547205,-113.522448,Breadland Organic Whole Grain Bakery,T5K 2T7,23,4.0,AB
57,"1150 Plymouth Meeting Mall, Ste 2230",DQ7PyYlp2bX96WZa7TcaWQ,"Ophthalmologists, Eyewear & Opticians, Health ...",Plymouth Meeting,1,40.116179,-75.282525,LensCrafters,19462,8,2.5,PA


In [139]:
merged_df.shape

(6114275, 24)